In [1]:
from os import environ

# environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HF_HOME"] = "/mnt/workspaces/hf_home"
environ["HTTP_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["HTTPS_PROXY"] = "http://clash.internal.moqi.ai:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"


from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY
from retrieval_qa_benchmark.utils.profiler import PROFILER

print(str(REGISTRY))

Datasets:
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	add_myscale_retrieval
LLMs:
	remote-llm
	gpt35
	chatgpt35
Evaluators:
	mcma
	mcsa


In [2]:
import yaml
from retrieval_qa_benchmark.utils.config import load
config = load(open("../config/mmlu-llama2-remote-retrieval-base.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
print(evaluator.transform)

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 3877.69it/s]
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fangruil/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fangruil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fangruil/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/fangruil/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


load dataset...


Resolving data files:   0%|          | 0/253 [00:00<?, ?it/s]

load index...
load mpnet model...
load Colbert model...
entry_id='0' chain={'0': AddMyScaleRetrievalTransform(children=(None, None), sep_chr='\n', prompt_prefix='Context:', num_filtered=100, num_selected=5, with_title=True, rank_dict={'mpnet': 30, 'bm25': 40}, index_path='/mnt/workspaces/yongqij/evals/data/indexes/Cohere_mpnet/IVFSQ_L2.index', embedding_name='paraphrase-multilingual-mpnet-base-v2', dataset_name=['Cohere/wikipedia-22-12-en-embeddings'])}


In [3]:
for _ in range(10):
    data = evaluator.transform(evaluator.dataset[0])
# print(evaluator.transform(evaluator.dataset[0]).raw_question)
evaluator.dataset.eval_set = evaluator.dataset.eval_set[:5]
data

QARecord(id='4e8db4a928333356588d2b89477e5d5bcb95532cd8afa355456ac94e13fd1849', question='Unlike most other early civilizations, Minoan culture shows little evidence of:', answer='conspicuous consumption by elites.', type='MCSA', context=['Minoan art | It was influenced by the neighbouring cultures of Ancient Egypt and the ancient Near East, which had produced sophisticated urban art for much longer, but the character of the small but wealthy mercantile Minoan cities was very different, with little evidence of large temple-based religion, monarchs, or warfare, and "all the imaginative power and childlike freshness of a very young culture". All these aspects of the Minoan culture remain rather mysterious. Sinclair Hood described an "essential quality of the finest Minoan art, the ability to create an atmosphere of movement and life although following a set of highly formal conventions".', 'Minoan civilization | Very little is known about the forms of Minoan government; the Minoan langua

In [4]:
print(evaluator.llm.convert_record(data))

The following are multiple choice questions (with answers) with context:

Context:
[1] Minoan art | It was influenced by the neighbouring cultures of Ancient Egypt and the ancient Near East, which had produced sophisticated urban art for much longer, but the character of the small but wealthy mercantile Minoan cities was very different, with little evidence of large temple-based religion, monarchs, or warfare, and "all the imaginative power and childlike freshness of a very young culture". All these aspects of the Minoan culture remain rather mysterious. Sinclair Hood described an "essential quality of the finest Minoan art, the ability to create an atmosphere of movement and life although following a set of highly formal conventions".
[2] Minoan civilization | Very little is known about the forms of Minoan government; the Minoan language has not yet been deciphered. It used to be believed that the Minoans had a monarchy supported by a bureaucracy. This might initially have been a numb

In [5]:
print(str(PROFILER))
PROFILER.clear()

	======== Profile Result ========
transform.base.profile                            	Count:     334	AvgTime:37542.42 ms
dataset.mmlu.profile                              	Count:      10	AvgTime:   13.76 ms
database.MyScaleSearch.emb_filter.profile         	Count:      10	AvgTime:406518.15 ms
database.MyScaleSearch.hybrid_search.profile      	Count:      10	AvgTime:845090.22 ms
database.MyScaleSearch.filtered_hybrid_search.profile	Count:      10	AvgTime:1251711.42 ms
database.MyScaleSearch.profile                    	Count:      10	AvgTime:1251786.47 ms
transform.add_myscale_retrieval.profile           	Count:      10	AvgTime:1251982.38 ms
transform.TransformChain.profile                  	Count:      10	AvgTime:1252203.42 ms



In [6]:
matched = evaluator()

Evaluating: 100%|██████████| 5/5 [01:11<00:00, 14.22s/it]
2023-08-11 10:37:38.549 | INFO     | retrieval_qa_benchmark.protocols.base:__call__:87 - Evaluation finished! Executed Evaluator:<class 'retrieval_qa_benchmark.protocols.mcsa.MCSAEvaluator'> on Dataset:cais/mmlu.prehistory-test with Model:llama2-13b-chat. Accuracy: 20.00%


In [7]:
print(str(PROFILER))

	======== Profile Result ========



In [7]:
with open("results.with-retrieval.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [ ]:
from retrieval_qa_benchmark.schema import QAPrediction

mismatched = [pred for pred in matched[1] if pred.matched]

In [ ]:
with open("results.test.jsonl", "w") as f:
    f.write("\n".join([r.model_dump_json() for r in mismatched]))